# 1. Introduction

We will be buidling a library that performs Automatic Differentiation (AD). Any client can install and use the library for personal or professional use and obtain an estimate of the value of the derivative (or gradient / Jacobian in higher dimension) of the function provided at the data point given as argument.

Performing fast differentiation and obtaining derivatives is absolutely necessary as it is a skill needed for a lot of real life applications. Indeed, most of systems modeling use differential equations to describe a behaviour and these equations require to take derivatives of sometimes complex function. Also, taking the gradient of a function at a given point and cancel it is the most effective way (at least analytically) to find the extrema of a function. Computing the values of extrema is a key feature to optimize a function and the processes it represents.

### Feedback

*Would have been nice to see more about why do we care about derivatives anyways and why is GrADim a solution compared to other approaches?*

Obtaining the derivative a function is a skill needed for a lot of real life applications. Indeed a lot of models used in different parts of science use differential equations to describe a behavior. As these equations contain explicitly written derivatives, it is important to know how to take them to solve the equations and have a good description of our system.

Even for problems where no derivative is explicitly written it is useful to know how to use them. For convex optimization problems, the global optimum we are looking for is located at the only point where the gradient of the function is null. For more complex cases, taking the derivative of a quantity is an important step of algorithms like Black Box Variational Inference for Bayesian neural networks or Hamiltonian Monte Carlo to obtain samples from any probability distribution.

With GrADim, we offer a way to compute effectively the derivative of a function using forward and reverse mode (see more details below). Compared to naive methods that could be used to compute a derivative, GrADim will be more precise as it will compute the exact numeric derivatives and not estimations. Also, it will allow the user to access to the computational graph of the function and to see the derivation process step by step. In that way, they will be able to use a tool which is not a black box and which they can easily understand.

# 2. Background 

We will provide a brief background to motivate our implementation of Automatic Differentiaion.

### 1. Intro to AD

AD is a way to obtain the value of the derivative of a function $f$ at a point $X$. The objective is to obtain a method providing more precise values than the naive estimators using Taylor expansion. Such estimators require fine tuning of parameters in order to give an approximation which is close enough to the truth value but which does not fail because of the floating point approximation.

### 2. Chain Rule

The Chain Rule is the key element of AD. Indeed we can decompose recursively a function $f$ into elementary components. For example, if we consider the function $f(x, y) = cos(x+y) \times sin(x-y)$, we can write it $f(x,y) = prod(cos(sum(x,y)), sin(difference(x, y))))$. Although unclear for a human eye, such a function is easier to derive by a machine using the chain rule:

$\frac{\partial f}{\partial x} = \frac{\partial f}{\partial u}\frac{\partial u}{\partial x} + \frac{\partial f}{\partial v}\frac{\partial v}{\partial x}$

In other words, you can compute the derivative of a function with respect to a variable by computing recursively the derivatives of each of the components and the derivative of the main function with respect to its components.

### Evaluation graph

When we can write a function as a series of simple components, we can obtain its evaluation graph. Here would be the evaluation graph for the example function provided above. 



![alt](evaluation_graph.PNG)

We also have the following evaluation table

|trace|elem operation|value of the function (as a function of $(x,y)$)|elem derivative|$\nabla_x$|$\nabla_y$|
|--|--|--|--|--|--|
|$u_{-1}$|$u_{-1}$|$x$|$\dot{u}_{-1}$|$1$|$0$|
|$u_0$|$u_0$|$y$|$\dot{u}_0$|$0$|$1$|
|$u_1$|$u_{-1} + u_0$|$x+y$|$\dot{u}_{-1} + \dot{u}_0$|$1$|$1$|
|$u_2$|$u_{-1} - u_0$|$x-y$|$\dot{u}_{-1} - \dot{u}_0$|$1$|$-1$|
|$u_3$|$cos(u_1)$|$cos(x+y)$|$-\dot{u}_1sin(u_1)$|$-sin(x+y)$|$-sin(x+y)$|
|$u_4$|$sin(u_2)$|$sin(x-y)$|$\dot{u}_2cos(u_2)$|$cos(x-y)$|$-cos(x-y)$|
|$u_5$|$u_3u_4$|$cos(x+y)sin(x-y)$|$\dot{u}_3u_4+u_3\dot{u}_4$|$-sin(x+y)sin(x-y) + cos(x+y)cos(x-y)$|$-sin(x+y)sin(x-y)-cos(x+y)cos(x-y)$|

### Feedback

*Good start to the background. Going forward, we would like to see more discussion on automatic differentiation. How do forward mode and reverse mode work? We would also like to see more discussion on what forward mode actually computes (Jacobian-vector product), the "seed" vector, and the efficiency of forward mode.*

If $f$ is a function with $m$ inputs and $n$ outputs, forward mode is a way to compute the partial derivatives of $f$ with respect to one of its input. To do that, we start from the roots of the evaluation tree (ie the inputs), compute the partial derivatives of all the inputs with respect to the selected one and go down the tree layer by layer computing the partial derivative of each node with respect to its parents. For example, if $g$ is an elementary function and in the evaluation graph $v_i = g(v_j)+v_k$, the partial derivative of $v_i$ will be $\dot{v}_i = \dot{v}_jg'(v_j)+\dot{v_k}$. If we plug the values of $\dot{v}_j$ and $\dot{v}_k$ computed before, we find the value of $\dot{v}_i$.

The direction (ie the vector) with respect to which the derivative is computed in the forward mode is called the seed vector. The forward mode AD allows to compute the product between the Jacobian of the function and this seed vector. This computation is a linear complexity of the number of states in the computational graph.


The reverse mode is a way to compute the partial derivatives of an output of $f$ with respect to all the inputs. To do that we start by the leaves of the graph (ie the outputs), compute the partial derivatives with respect to their parents and do the same by going up in the evaluation graph. For example, if we already know the value of the partial derivative $\frac{\partial f_i}{\partial v_j}$ and we know that $v_j = g(v_k)$ where $g$ is an elementary function, we can use the chain rule to write $\frac{\partial f_i}{\partial v_k} = \frac{\partial f_i}{\partial v_j}\times \frac{\partial v_j}{\partial v_k} = \frac{\partial f_i}{v_j}\times g'(v_k)$. 


# 3. How to Use GrADim

We will briefly demonstrate how to use our package

### Installing and importing the package

A user can install the package using:

In [ ]:
>>> pip install GrADim

The package can be imported using the following command:

In [ ]:
>>> import GrADim as ad

### Using the package

An instance of AD object can be created and used to find the derivative for the required point:

In [ ]:
>>> import GrADim as ad
>>> import numpy as np

>>> def fun(x,y):
    return np.cos(x+y)*np.cos(x-y)

>>> autodiff = ad.derivative(fun)
>>> autodiff.fwd_pass((1,1))
>>> autodiff.rev_pass()

# 4. Software Organization 

1. Directory Structure
    The package directory would look like the following tree.

    File README.md will contain instructions and provide examples using the package. License folder will contain relevant licensing information. File requirements.txt will contain details for package to be distributed.




In [ ]:
master
├── LICENSE
├── README.md     
├── docs
│   ├── ...
│   └── ...
├── requirements.txt
├── travis.yml
├── GrADim
│   ├── ...
│   └── ...
├── Tests
    ├── ...
    └── ...

2. The basic modules
    GrADim is the main module for the library where callable submodules used for automatic differentation will be stored. Test will contain testing submodules for GrADim.

3. Test Suite
    We will use both TravisCI and CodeCov. Travis CI will be used to detect when a commit has been made and pushed onto Github, and then try to build the project and run tests each time it happens. Codecov will be used to provide an assessment of the code coverage.

4. Package distribution
    PyPi will be used to distribute the package based on the format in the directory tree.

5. Framework considerations
    Framework is not currently considered as current implementation is not as complicated. Should the implementation complexity evolves, we will then consider implementing a framework.

# 5. Implementation

We will now go into some implementation considerations

### 1. Data Structures
We will use floats if the user asks for a single output. If the user asks for a number of outputs for several inputs, we will use arrays. Different classes are defined for different input options as explained below.

### 2. Classes

We will have one class for the generic type of the automatic differentiation object.

Inside this generic class, we have one method for calculating derivative values, including a number of if-else statements:
- For the function input, we have one if-else block to check whether it contains matrices. If yes, the program will call the method of matrix operations for differentiation. Otherwise, call the usual automatic differentiation method.
- For the number of input variables, we have one if-else block to check if it is larger than 1. If univariate, the program will implement the differentiation function with only one variable. If multivariate, the program calls the multivariate differentiation method.
- For univariate differentiation, we have one nested if-else block to check whether the input variable is a single value or an array of values. If input is a single number, the program will implement simple differentiation. Otherwise, the input is an array, then the program will iterate through the array of values for simple differentiation.
- For multivariate differentiation, we have a nested if-else block to check whether the input variable is a matrix of  values. If it is a vector, the program will implement multivariate automatic differentiation. Otherwise, the input values are in matrix form, then the program will iterate through each vector and call the multivariate automatic differentiation.
- For the function implemented, we have one if-else block to check if the function contains matrices. If it contains matrices, the program will implement the matrix version of differentiation, in univariate or multivariate form, depending on the number of input variables. Otherwise, the program will implement the usual form of differentiation that do not invlove matrix multiplication.

For automatic differentiation, an elementary operation would be something like:

In [ ]:
def sin(self, x):
    if x is a vector:
        for variable in vector:
            self.partial_variable = self.cos(x)
            self.val = self.sin(x)

    else:
        self.der = self.cos(x)
        self.val = self.sin(x)

We have one subclass of differentiation that implements the most basic form of differentiation: input has one variable and one function, and the output is one value representing the function derivative. 
We have one subclass of differentiation, partial differentiation, that handles the input function with more than one variables.
We have one subclass of differentiation for automatic differentiation that covers the case where the input function is in matrix form.

Methods that all classes share include hard coding the basic differentiations, (e.g., $\frac{dc}{dx}$ = 0 where c is a constant, $\frac{d sin(x)}{dx}$ = cos(x), $\frac{dx^a}{dx}$ = ax<sup>a-1</sup>etc.) and chain rule. For multivariate differentiation, methods are defined to calculate partial derivatives ($\frac{\partial (xy)}{\partial x} = y, \frac{\partial (xy)}{\partial y} = x$). When necessary, we will overwrite the pre-defined methods so that the program would do the differentiation.

Name attributes contain function values and derivatives.

### 3. External dependencies
We will use numpy elementary operations (e.g., sin, cos, log, exp, tan, sinh, cosh, tanh, arcsin, arctan, etc.). We will use scipy is used mainly for more complicated matrix algebra. If the input function is passed in via scipy, we may use a scipy dependency in implementation.

### 4. Other considerations 
We will consider differentiation in polar coordinate as well as that in cartesian coordinate.

### Feedback

Will you implement operator overloading methods? If so, what methods will you overload?


Yes, we will be implementing operator overloading to natural operators such as addition, multiplication, subtraction, and raising to a power etc.

# 6. Licensing 

We would want our library to be open source and accessible to everyone. We would be okay with others mmodifying our code and if the modifications are distributed, and if it is used in comercial softwares, with proper acknowledgement. So we opt for MIT copyright license. 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=077351ea-f043-41bb-a867-a9ce6280de54' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>